In this example i'll finetune openai CLIP on the PASCAL VOC 2012 Dataset (https://www.kaggle.com/datasets/gopalbhattrai/pascal-voc-2012-dataset) 

In [1]:
# code adapted from: https://github.com/openai/CLIP/issues/83 

import clip 
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torchvision 
from torch.utils.data import Dataset, DataLoader 
from PIL import Image 

import albumentations as A 
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt 

import random 

seed = 10 
random.seed(seed)
np.random.seed(seed) 
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)


In [2]:

EPOCH = 4 
BATCH_SIZE = 16 


transform = A.Compose([ # yup i'm using albumentations not torchvision for these transforms haha though ofc torchvision works too 
    A.HorizontalFlip(p=0.5), 
    A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1)), 
    #A.RandomRain(), 
])



device = "cuda:0" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.
print("DEVICE:", device)
model, preprocess = clip.load("ViT-L/14",device=device,jit=False) #Must set jit=False for training



def train_preprocess(x:Image.Image): # might not be the optimal way but haha 
    x = transform(np.array(x)) 
    return preprocess(Image.fromarray(x['image'])) 


# this is done first, so that we can get the preprocess() function used by CLIP and stuff before data loading 


DEVICE: cuda:0


In [3]:
# data preparation 

class VOCDDataset(Dataset):
    surrounding_pixels = 0

    vocd_train = torchvision.datasets.VOCDetection('./data/', image_set='train', download=True) 
    vocd_test = torchvision.datasets.VOCDetection('./data/', image_set='val', download=True)


    def assign_indices(self): 

        if self.mode == 'train': 
            data_len = len(VOCDDataset.vocd_train)
        elif self.mode == 'test': 
            data_len = len(VOCDDataset.vocd_test)

        idxs = list(range(data_len)) 
        if self.shuffle: 
            random.shuffle(idxs) 

        self.idxs = idxs 

        self.length = 0 
        self.poss = [] 
        for i in idxs: 
            self.poss.append( self.length ) 
            if self.mode == 'train': 
                num_objs = len(VOCDDataset.vocd_train[i][1]['annotation']['object']) # number of objects 
            elif self.mode == 'test': 
                num_objs = len(VOCDDataset.vocd_test[i][1]['annotation']['object'])
            self.length += num_objs 
        self.poss.append(self.length) 

        #print("POSS:", self.poss)
        
        self.next_i = 0 



    def __init__(self, mode='train', shuffle=True):
        assert mode in ['train', 'test',], 'Only train and test sets are available'
        self.mode = mode 


        self.shuffle = shuffle 

        self.assign_indices()

    
    def __len__(self):
        return self.length 

    def __getitem__(self, in_idx):
        idxset = False 
        if (self.poss[self.next_i] <= in_idx): 
            if (in_idx < self.poss[self.next_i+1]): 
                idx = self.next_i 
                idxset = True 
            elif (in_idx < self.poss[self.next_i+2]): 
               self.next_i += 1 
               idx = self.next_i 
               idxset = True 
        
        if (not idxset): 
            # binary search out idx 
            left=0 

            right=len(self.poss)-1
            
            while (right-left > 1): 
                mid = (left+right)//2 
                if (self.poss[mid] > in_idx): 
                    right = mid 
                else: 
                   left = mid 

            if (in_idx > self.poss[right]): 
               self.next_i = right 
               idx = right 
            else: 
               self.next_i = left 
               idx = left 

        #print("IN_IDX {} > POSS[{}] ({})".format(in_idx, idx, self.poss[idx])) 
        
        if self.mode == 'train': 
            image, y = VOCDDataset.vocd_train[self.idxs[idx]]
        elif self.mode == 'test': 
            image, y = VOCDDataset.vocd_test[self.idxs[idx]]

        objects = y['annotation']['object'] 
        # objects[i]['name], objects[i]['bndbox'] will be dict of {'xmin', 'ymin', 'x}
        obj = objects[in_idx-self.poss[idx]]

        crop_bbox = self.get_crop_bbox(obj['bndbox'], *image.size)

        image = image.crop(crop_bbox) 
        image = transform(image=np.array(image))['image']

        '''plt.figure(figsize=(20,10)) 
        plt.axis('off') 
        plt.imshow(image)
        plt.show() 
        print(image) ''' 

        image = preprocess(Image.fromarray(image)) # Image from PIL module

        caption = obj['name']
        return image, clip.tokenize(caption)[0] 
    
    def get_crop_bbox(self, bndbox, maxx, maxy): 
        
        a = max(0, int(bndbox['xmin'])-VOCDDataset.surrounding_pixels) 
        b = max(0, int(bndbox['ymin'])-VOCDDataset.surrounding_pixels)
        c = min(maxx, int(bndbox['xmax'])+VOCDDataset.surrounding_pixels) 
        d = min(maxy, int(bndbox['ymax'])+VOCDDataset.surrounding_pixels)
        return (a, b, c, d) 

dataset = VOCDDataset(mode='train')
train_dataloader = DataLoader(dataset,batch_size = BATCH_SIZE) #Define your own dataloader


Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data/
Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data/


In [4]:

#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 


if device == "cpu":
  model.float()
else :
  clip.model.convert_weights(model) # Actually this line is unnecessary since clip by default already on float16



In [5]:

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-7,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

print("DEVICE:", device)



# training process 
for epoch in range(EPOCH):
    print("EPOCH", epoch)
    for batch in train_dataloader :
        #print("batch")
        optimizer.zero_grad()

        images, texts = batch 

        #print("IMAGES:", images) 
        #print("TEXTS:", texts)
        
        images = images.to(device)
        texts = texts.to(device)
        
        logits_per_image, logits_per_text = model(images, texts)

        ground_truth = torch.arange(len(images),dtype=torch.long,device=device)

        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        total_loss.backward()
        if device == "cpu":
            optimizer.step()
        else : 
            convert_models_to_fp32(model)
            optimizer.step()
            clip.model.convert_weights(model)



    # save model 

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
        }, "clip_models/model_"+str(epoch)+".pt") 



DEVICE: cuda:0
EPOCH 0
EPOCH 1
EPOCH 2
EPOCH 3
